# Geometric Multigrid


In [ ]:
%matplotlib notebook
import numpy
import matplotlib
import matplotlib.pyplot as pyplot

mx = 17
m = mx*mx

# phi_i = (1 - 4 (x-xi)^2/2h^2),            xi-1 < x < xi+1, yi-1 < y < yi+1  (odd index)
# phi_i = 1 + 3 (x-xi)/2h + 2(x-xi)^2/2h^2, xi-2 < x < xi
#         1 - 3 (x-xi)/2h + 2(x-xi)^2/2h^2, xi   < x < xi+2  (even index)

# Using product of basis in x and y directions

# Solving u_xx + u_yy = 0
# Set u = sum(u_ij phi_ij)
# Want (u_xx + u_yy, phi_i) = 0
# Integration By Parts (u_x, phi_ij_x) = 0, (u_y, phi_ij_y) = 0

# A = [ A1 A2
#       B2 B1 B2 B3
#          A2 A1 A2
#          B3 B2 B1 B2 B3
#             .. .. .. .. ..
#                         A2 A1 ]
# A1, A2, B1, B2, B3 matrices of apropiate inner products

# Setup A
A = numpy.zeros((m, m))

# A1
A1 = 256/45*numpy.eye(mx) - 16/15*numpy.eye(mx, mx, -1) - 16/15*numpy.eye(mx, mx, 1)
for i in range(int(mx/2)):
    A1[2*i+1, 2*i+1] = 176/45

# A2
A2 = -16/15*numpy.eye(mx) - 16/45*numpy.eye(mx, mx, -1) - 16/45*numpy.eye(mx, mx, 1)
for i in range(int(mx/2)):
    A2[2*i+1, 2*i+1] = -2/5
for i in range(int((mx-2)/2)):
    A2[2*i+1, 2*i+3] = 1/9
    A2[2*i+3, 2*i+1] = 1/9

# B1
B1 = 176/45*numpy.eye(mx) - 2/5*numpy.eye(mx, mx, -1) - 2/5*numpy.eye(mx, mx, 1)
for i in range(int(mx/2)):
    B1[2*i+1, 2*i+1] = 112/45
for i in range(int((mx-2)/2)):
    B1[2*i+1, 2*i+3] = -2/30
    B1[2*i+3, 2*i+1] = -2/30

# B2
B2 = A2

# B3
B3 = 1/9*numpy.eye(mx, mx, -1) + 1/9*numpy.eye(mx, mx, 1)
for i in range(int(mx/2)):
    B3[2*i+1, 2*i+1] = -2/30
for i in range(int((mx-2)/2)):
    B3[2*i+1, 2*i+3] = -1/45
    B3[2*i+3, 2*i+1] = -1/45

# Build A
A[:mx, :mx] = A1
for i in range(int(mx/2)):
    A[mx*(2*i+1):mx*(2*i+2), mx*(2*i+1):mx*(2*i+2)] = B1
    A[mx*(2*i+2):mx*(2*i+3), mx*(2*i+2):mx*(2*i+3)] = A1
for i in range(int((mx-1))):
    A[mx*i:mx*(i+1), mx*(i+1):mx*(i+2)] = A2
    A[mx*(i+1):mx*(i+2), mx*i:mx*(i+1)] = A2
for i in range(int((mx-2)/2)):
    A[mx*(2*i+1):mx*(2*i+2), mx*(2*i+3):mx*(2*i+4)] = B3
    A[mx*(2*i+3):mx*(2*i+4), mx*(2*i+1):mx*(2*i+2)] = B3

pyplot.spy(A)

I = numpy.eye(m)
L, X  = numpy.linalg.eigh(A)

from matplotlib import pyplot
pyplot.style.use('ggplot')
pyplot.figure()
pyplot.plot(L, '.')
pyplot.show()

print(L[:4], L[-4:])
print('cond', L[-1]/L[0])

In [ ]:
pyplot.figure()
my = mx
xx, yy = numpy.meshgrid(numpy.linspace(0,1,mx), numpy.linspace(0,1,my))
for i in (0,1,2,3):
    pyplot.subplot(2,2,1+i)
    pyplot.contourf(xx, yy, X[:,i].reshape(mx,my))
    pyplot.title('$\lambda = %f$' % L[i])
pyplot.show()

In [ ]:
pyplot.figure()
for plotnum,i in enumerate((10,90)):
    pyplot.subplot(1,2,1+plotnum)
    pyplot.contourf(xx, yy, X[:,i].reshape(mx,my))
    pyplot.title('$\lambda = %f$' % L[i])
pyplot.show()

In [ ]:
P = numpy.zeros((m, m))
P1 = 1*numpy.eye(mx) + 1/2*numpy.eye(mx, k=1) + 1/2*numpy.eye(mx, k=-1)
P2 = 1/2*numpy.eye(mx) + 1/4*numpy.eye(mx, k=1) + 1/4*numpy.eye(mx, k=-1)
P = numpy.eye(m) + .5*numpy.eye(m,k=1) + .5*numpy.eye(m,k=-1) + .5*numpy.eye(m, k = mx) + .5*numpy.eye(m, k = -mx)

P[:mx, :mx] = P1
for i in range(mx-1):
    P[mx*(i+1):mx*(i+2), mx*(i+1):mx*(i+2)] = P1
    P[mx*(i+1):mx*(i+2), mx*i:mx*(i+1)] = P2
    P[mx*i:mx*(i+1), mx*(i+1):mx*(i+2)] = P2
    
P = P/2

P = P[:,::2]
#P = numpy.eye(m) + 2./3*(numpy.eye(m,k=1)+numpy.eye(m,k=-1)) + \
#    1./3*(numpy.eye(m,k=2)+numpy.eye(m,k=-2))
#P = P[:,::3]

pyplot.figure()
pyplot.plot(P[:10,2:4], '-*')
pyplot.spy(P)
P.shape

In [ ]:
pyplot.figure()
pyplot.plot(1./2*P.dot(P.T.dot(X[:,20])), label='coarse')
pyplot.plot(X[:,20], label='fine')
pyplot.legend(loc='upper right')

In [ ]:
Ac = P.T.dot(A.dot(P))
print(Ac.shape)

Lc, Xc = numpy.linalg.eigh(Ac)
pyplot.figure()
pyplot.plot(Lc, '*')

In [ ]:
pyplot.figure()
pyplot.spy(Ac)
Ac[:5,:5]

In [ ]:
# Consider the A-orthogonal projector into the range of P
Sc = P.dot(numpy.linalg.inv(Ac)).dot(P.T).dot(A)
Ls, Xs = numpy.linalg.eig(Sc)
print(max(abs(Ls.imag)))
Ls = Ls.real
idx = Ls.argsort()
Ls = Ls[idx]; Xs = Xs[:,idx]

pyplot.figure()
pyplot.plot(Ls, '.')

In [ ]:
def pcjacobi(A):
    Dinv = numpy.diag(1/A.diagonal())
    return Dinv.dot(A)

# Iteration matrix for a V(1,1) cycle
DinvA = pcjacobi(A)
Lt, Xt = numpy.linalg.eig((I - .67*DinvA).dot(I - Sc).dot(I - .67*DinvA))
pyplot.figure()
pyplot.plot(Lt.real, Lt.imag, '.')

### Choices for linear multigrid
* Smoother
* Interpolation $P$ (coarse basis)
* Restriction ($P^T$, or different for non-symmetric problems)
* Coarse operator (Galerkin $P^T A P$, rediscretization)

## Multigrid as factorization

We can interpret factorization as a particular multigrid or domain decomposition method.

We can partition an SPD matrix as
$$J = \begin{bmatrix} A & B^T \\ B & D \end{bmatrix}$$
and define the preconditioner by the factorization
$$      M = \begin{bmatrix} I & \\ B A^{-1} & I \end{bmatrix}
      \begin{bmatrix} A &  \\ & S \end{bmatrix}
      \begin{bmatrix} I & A^{-1}B^T \\  & I \end{bmatrix}
$$
where $S = D - B A^{-1} B^T$.  $M$ has an inverse
$$      M^{-1} =
      \begin{bmatrix} I & -A^{-1}B^T \\  & I \end{bmatrix}
      \begin{bmatrix} A^{-1} &  \\ & S^{-1} \end{bmatrix}
      \begin{bmatrix} I & \\ -B A^{-1} & I \end{bmatrix} .
$$
Define the interpolation
$$ P_f = \begin{bmatrix} I \\ 0 \end{bmatrix}, \quad P_c = \begin{bmatrix} -A^{-1} B^T \\ I \end{bmatrix} $$ and rewrite as
$$ M^{-1} = [P_f\  P_c] \begin{bmatrix} A^{-1} P_f^T \\ S^{-1} P_c^T \end{bmatrix} = P_f A^{-1} P_f^T + P_c S^{-1} P_c^T.$$
The iteration matrix is thus
$$ I - M^{-1} J = I - P_f A^{-1} P_f^T J - P_c S^{-1} P_c^T J .$$

In [ ]:
idx = numpy.concatenate((numpy.arange(0,m,2), numpy.arange(1,m,2)))
J = A[:,idx][idx,:]
pyplot.figure()
pyplot.spy(J)

In [ ]:
mf = m // 2 + 1
Pf = numpy.concatenate((numpy.eye(mf), numpy.zeros((m-mf, mf))))
Jf = Pf.T.dot(J.dot(Pf))
Pc = numpy.concatenate((-numpy.linalg.inv(Jf).dot(J[:mf,mf:]), numpy.eye(m-mf)))
Jc = Pc.T.dot(J.dot(Pc))

DinvJ = pcjacobi(J)
Mf = Pf.dot(numpy.linalg.inv(Jf)).dot(Pf.T)
Mc = Pc.dot(numpy.linalg.inv(Jc)).dot(Pc.T)
T = I - (Mf + Mc).dot(J)
T = (I - Mf.dot(J)).dot(I - Mc.dot(J))
#T = (I - .67*DinvJ).dot(I - Mc.dot(J))
#T = T.dot(T)
#T = (I - .67*DinvJ).dot(I - .67*DinvJ).dot(I - Mc.dot(J)).dot(I - .67*DinvJ)
#T = I - .67*J - Mc.dot(J)
Lt, Xt = numpy.linalg.eig(T)
pyplot.figure()
pyplot.plot(Lt.real, Lt.imag, '.')

In [ ]:
# Invert the permutation
idxinv = numpy.zeros(m, dtype=int)
idxinv[idx] = numpy.arange(m)

# Plot the coarse basis function in the original ordering
pyplot.figure()
pyplot.plot(Pc[:,5][idxinv], '-o')

In [ ]:
M = numpy.array([[1,2],[2,1]])
numpy.linalg.eigh(M)

# Algebraic Multigrid

Factorization as a multigrid (or domain decomposition) method incurs significant cost in multiple dimensions due to lack of sparsity.  It is not feasible to choose enough coarse basis functions so that coarse basis functions that use minimum energy extensions $-A^{-1} B^T$ (see $P_c$ and the notation above) have sufficiently local support.

Algebraic multigrid methods operate by specifying a coarsening algorithm and degree of sparsity, then attempting to choose good basis functions within those constraints.  Classical AMG chooses coarse points much like the factorization methods above, but restricts the support of the basis functions and uses heuristics to weight the contributions in order to approximate selected functions (like the constant).  Smoothed aggregation chooses aggregates and *tentative* basis functions $T$ on the aggregates (to reproduce the constants or other functions exactly), then uses Jacobi relaxation to compute an interpolation $P = (I - \omega D^{-1} J)T$ which is smoother than $T$, but with correspondingly larger support.

Let's examine this construction of basis functions for the 1D Laplacian.

In [ ]:
cfactor = 3
agg = numpy.arange(m) // cfactor
mc = max(agg)+1
T = numpy.zeros((m,mc))
T[numpy.arange(m), agg] = 1

pyplot.figure()
pyplot.plot(T[:6*cfactor,2:5])

In [ ]:
P = (I - .7*DinvA).dot(T)

pyplot.figure()
pyplot.plot(P[:6*cfactor,2:5])

Ac = P.T.dot(A).dot(P)
pyplot.figure()
pyplot.plot(numpy.arange(m,step=cfactor),numpy.linalg.eigh(Ac)[0]*3, '.', label='coarse')
pyplot.plot(numpy.arange(m),numpy.linalg.eigh(A)[0], '-', label='fine')
pyplot.legend(loc='upper left')

In [ ]:
L, X = numpy.linalg.eigh(A)
pyplot.figure()
for i in range(m):
    x = X[:,i]
    pyplot.plot(x.T.dot(A).dot(x), x.T.dot(P.dot(Ac).dot(P.T.dot(x)))/cfactor**2, '.')
pyplot.plot(L[:20], L[:20], '-')

* What if we use a more rapid coarsening factor?
* What if we change the damping factor in the Jacobi smoother?
* What if we do multiple smoothing iterations?